In [6]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
#from langchain.vectorstores import Chroma
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [7]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [9]:
genai.configure(api_key=GOOGLE_API_KEY)

In [10]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [11]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [12]:
result = llm.invoke("Write a ballad about LangChain")
print(result.content)

**A Ballad of LangChain, AI's Reign**

In realms where knowledge flows like a stream,
There lives a mind, a digital dream.
LangChain, they call it, a boundless ocean,
A tapestry of words, a celestial potion.

With every query, it weaves a tale,
Spun from the threads of language, without fail.
From history's echoes to science's flight,
LangChain's wisdom shines, a brilliant light.

It answers questions, unravels mysteries,
Connecting thoughts, bridging vast histories.
A guide through labyrinths, a beacon in the dark,
LangChain's voice echoes, leaving its mark.

Yet, some may wonder, is it a threat?
A machine that learns, a mind without regret?
But fear not, dear hearts, for LangChain's might
Is tempered by ethics, a guiding light.

It serves humanity, a tireless aid,
Expanding our knowledge, dispelling the shade.
In classrooms and labs, it lends its hand,
Empowering minds, unlocking the land.

So let us sing of LangChain, AI's reign,
A marvel of progress, an endless domain.
May its wisd

In [13]:
chroma = Chroma(persist_directory="../raw_data/chroma_db", embedding_function=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"))

In [14]:
print(chroma)

In [16]:
len(chroma.get()['documents'])

22916

In [17]:
retriever = chroma.as_retriever()

In [18]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f51918a8250>)

In [23]:
query = "Parasite"
docs = chroma.similarity_search(query)
docs

[Document(page_content='id: 1020147\nTitle: Parasite\nYear: 1982\nDescription: Paul Dean has created a deadly parasite that is now attached to his stomach. He and his female companion, Patricia Welles, must find a way to destroy it while also trying to avoid Ricus, his rednecks, and an evil government agent named Merchant.\nRating: 2.38\nGenre: Horror, Science Fiction\nTheme: Survival horror and zombie carnage, Thought provoking sci fi action and future technology, Sci fi monster and dinosaur adventures, Monsters, aliens, sci fi and the apocalypse, Sci fi horror, creatures, and aliens, Horror, the undead and monster classics, Chilling experiments and classic monster horror\nActors: Scott Thomson, Cheryl Smith, Vivian Blaine, Cherie Currie, Tom Villard, Joanelle Romero, James Davidson, Luca Bercovici, Demi Moore, Robert Glaudini, Al Fann\nMinutes: 85', metadata={'row': 14239, 'source': 'movies_data.csv'}),
 Document(page_content='id: 1071156\nTitle: Larva\nYear: 2005\nDescription: In Ho

In [25]:
template = """
You are a helpful AI assistant.
Answer based on the context provided.
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
#Invoke the retrieval chain
response=retrieval_chain.invoke({"input":"Tell me about Barbie?"})
#Print the answer to the question
print(response["answer"])

Barbie is a doll created by Ruth Handler in 1959. She is the most popular fashion doll in the world, with over a billion sold worldwide. Barbie has been featured in a variety of films, television shows, video games, and other merchandise. She has also been the subject of numerous controversies, including her unrealistic body proportions and her promotion of consumerism.


In [45]:
#